In [3]:
#for windows users:
#pip install bitermplus

#for mac users:
# xcode-select --install
# brew install libomp
# pip3 install bitermplus

# pip install tmplot
# pip install tomotopy

In [11]:
import bitermplus as btm
import numpy as np
import pandas as pd

import json 
import csv
import nltk
import unicodedata
import re
import sys
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim, spacy
from gensim.models.ldamulticore import LdaMulticore
import re
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [5]:
df = pd.read_csv("../pos_neg_india.csv")
df

,Unnamed: 0,index,Text,new_text,languages,translated,new_languages,pos_tagged_words,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,0,0,soniya mummy hoti toh covid me magic stick ghu...,soniya mummy hoti toh covid me magic stick ghu...,Unknown,soniya mummy hoti toh covid me magic stick ghu...,Unknown,"[('soniya', 'JJ'), ('mummy', 'NN'), ('hot', 'J...","['soniya', 'mummy', 'hot', 'toh', 'couid', 'ma...",['death'],['death'],subjective,0
1,12,12,is covid ki waja se buht saray logon ne apne p...,is covid ki waja se buht saray logon ne apne p...,Unknown,is covid ki waja se buht saray logon ne apne p...,Unknown,"[('is', 'VBZ'), ('couid', 'JJ'), ('ki', 'NN'),...","['couid', 'ki', 'raja', 'se', 'spray', 'logon'...","['hair', 'ply', 'vaccination']","['hair', 'ply', 'vaccination']",subjective,2
2,18,18,deaths due covid what is vaccination rate what...,deaths due covid what is vaccination rate what...,"ENGLISH, Unknown",deaths due covid what is vaccination rate what...,"ENGLISH, Unknown","[('deaths', 'NNS'), ('due', 'JJ'), ('couid', '...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2
3,28,28,vaccination against covid- 19 is necessity tod...,vaccination against covid- 19 is necessity tod...,ENGLISH,vaccination against covid- 19 is necessity tod...,ENGLISH,"[('vaccination', 'NN'), ('against', 'IN'), ('c...","['vaccination', 'couid', 'nineteen', 'necessit...","['necessity', 'today', 'scenario', 'hope', 'pr...","['necessity', 'today', 'scenario', 'problem']",subjective,2
4,33,33,very surprising that unvaccinated population &...,very surprising that unvaccinated population &...,ENGLISH,very surprising that unvaccinated population &...,ENGLISH,"[('very', 'RB'), ('surprising', 'JJ'), ('that'...","['surprising', 'vaccinated', 'population', 'yr...","['surprising', 'vaccinated', 'population', 'yr...","['population', 'yrs', 'offline', 'class', 'amp...",subjective,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37618,95179,95179,nature seems have already started working savi...,nature seems have already started working savi...,ENGLISH,nature seems have already started working savi...,ENGLISH,"[('nature', 'NN'), ('seems', 'VBZ'), ('have', ...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'generation', 'vaccinesaveslive', '...",subjective,2
37619,95180,95180,"dear , we #india 🇮🇳has helped this #world incl...","dear , we #india Indiahas helped this #world i...",ENGLISH,"dear , we #india Indiahas helped this #world i...",ENGLISH,"[('dear', 'IN'), ('we', 'PRP'), ('india', 'VBP...","['dear', 'india', 'indians', 'help', 'world', ...","['dear', 'help', 'world', 'include', 'kindness...","['world', 'kindnessmatter', 'vaccinesaveslive']",subjective,2
37620,95184,95184,our sincere request state central govt 🙏🙏. ple...,our sincere request state central govt folded_...,ENGLISH,our sincere request state central govt folded_...,ENGLISH,"[('our', 'PRP$'), ('sincere', 'JJ'), ('request...","['sincere', 'request', 'state', 'central', 'go...","['sincere', 'request', 'state', 'central', 'go...","['request', 'state', 'govt', 'share', 'vaccine...",subjective,2
37621,95185,95185,thanks your appreciation valuable feedback abo...,thanks your appreciation valuable feedback abo...,ENGLISH,thanks your appreciation valuable feedback abo...,ENGLISH,"[('thanks', 'NNS'), ('your', 'PRP$'), ('apprec...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'feedback', 'vaccine...",subjective,2


In [6]:
df.drop(columns = ['Unnamed: 0', 'Text', 'index', 'new_text', 'languages', 'new_languages', 'pos_tagged_words'], inplace = True)
df

,translated,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,soniya mummy hoti toh covid me magic stick ghu...,"['soniya', 'mummy', 'hot', 'toh', 'couid', 'ma...",['death'],['death'],subjective,0
1,is covid ki waja se buht saray logon ne apne p...,"['couid', 'ki', 'raja', 'se', 'spray', 'logon'...","['hair', 'ply', 'vaccination']","['hair', 'ply', 'vaccination']",subjective,2
2,deaths due covid what is vaccination rate what...,"['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2
3,vaccination against covid- 19 is necessity tod...,"['vaccination', 'couid', 'nineteen', 'necessit...","['necessity', 'today', 'scenario', 'hope', 'pr...","['necessity', 'today', 'scenario', 'problem']",subjective,2
4,very surprising that unvaccinated population &...,"['surprising', 'vaccinated', 'population', 'yr...","['surprising', 'vaccinated', 'population', 'yr...","['population', 'yrs', 'offline', 'class', 'amp...",subjective,0
...,...,...,...,...,...,...
37618,nature seems have already started working savi...,"['nature', 'seem', 'already', 'start', 'work',...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'generation', 'vaccinesaveslive', '...",subjective,2
37619,"dear , we #india Indiahas helped this #world i...","['dear', 'india', 'indians', 'help', 'world', ...","['dear', 'help', 'world', 'include', 'kindness...","['world', 'kindnessmatter', 'vaccinesaveslive']",subjective,2
37620,our sincere request state central govt folded_...,"['sincere', 'request', 'state', 'central', 'go...","['sincere', 'request', 'state', 'central', 'go...","['request', 'state', 'govt', 'share', 'vaccine...",subjective,2
37621,thanks your appreciation valuable feedback abo...,"['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'feedback', 'vaccine...",subjective,2


In [7]:
df['words_processed_all'].values[0][0]

'['

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [12]:
tokens_words_processed_all = list(sent_to_words(df['words_processed_all']))

In [13]:
from nltk.corpus import stopwords
stop_list = stopwords.words('english')
high_freq_known_words = ['covid','couid', 'pfizer', 'amp', 'people', 'covidvaccine', 'covishield', 'moderna', 'vaccine', 'astrazeneca', 'vaccinate', 'vaccinated', 'vaccination', 'vaccines']

stop_list.extend(high_freq_known_words)

test_words_only1 = ['not', 'vaccine', 'covid', 'and', 'vegan', 'cruelty', 'free', 'i', 'use', 'elf', 'cosmetics', 'every', 'day', 'and', 'i', 'love', 'it']
test_stopremoved = [w for w in test_words_only1 if w not in stop_list or w == 'not']
print(test_stopremoved)

['not', 'vegan', 'cruelty', 'free', 'use', 'elf', 'cosmetics', 'every', 'day', 'love']


In [14]:
stop_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [15]:
df['words_processed_all'] = tokens_words_processed_all

In [16]:
# remove stop words from tokenised_words_only

tokenised_stopremoved = [[w for w in each if w not in stop_list or w == 'not'] for each in tokens_words_processed_all]
tokenised_stopremoved

[['soniya',
  'mummy',
  'hot',
  'toh',
  'magic',
  'stick',
  'huma',
  'ke',
  'death',
  'kam',
  'kar',
  'deli',
  'aur',
  'rahul',
  'baba',
  'ape',
  'hadto',
  'se',
  'tayyar',
  'kake',
  'data'],
 ['ki',
  'raja',
  'se',
  'spray',
  'logon',
  'ne',
  'ape',
  'pardon',
  'ko',
  'khoja',
  'hai',
  'sare',
  'log',
  'ani',
  'jan',
  'se',
  'hath',
  'bethe',
  'agr',
  'ap',
  'susan',
  'se',
  'bacha',
  'chaste',
  'hair',
  'ply',
  'larva'],
 ['death',
  'due',
  'rate',
  'free',
  'safe',
  'sir',
  'gallian',
  'nikola',
  'ata',
  'tuck',
  'congress',
  'mubarak',
  'kashmir',
  'nha',
  'se',
  'kashmiri',
  'pandit',
  'ko',
  'braga',
  'mar',
  'gay',
  'harm',
  'parivartan',
  'rne',
  'bola',
  'gay',
  'wo',
  'mubarak',
  'dunny',
  'ke',
  'mubarak',
  'nai',
  'koi',
  'mubarak',
  'kashmir',
  'nha',
  'se',
  'kashmiri',
  'pandit',
  'ko',
  'braga',
  'mar',
  'gay',
  'harm'],
 ['nineteen',
  'necessity',
  'today',
  'scenario',
  'hoping

In [17]:
df['words_processed_all'].values[0][0]

'soniya'

In [18]:
df['tokenised_stopremoved'] = tokenised_stopremoved

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37623 entries, 0 to 37622
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   translated                         37623 non-null  object
 1   words_processed_all                37623 non-null  object
 2   words_processed_noun_adj_verb_adv  37623 non-null  object
 3   words_processed_noun               37623 non-null  object
 4   text_type                          37623 non-null  object
 5   sentiment                          37623 non-null  int64 
 6   tokenised_stopremoved              37623 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.0+ MB


In [20]:
df['sentiment'].value_counts()

0    20029
2    17594
Name: sentiment, dtype: int64

In [21]:
positive_df = df[df['sentiment'] == 2]

In [22]:
positive_df

,translated,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment,tokenised_stopremoved
1,is covid ki waja se buht saray logon ne apne p...,"[couid, ki, raja, se, spray, logon, ne, ape, p...","['hair', 'ply', 'vaccination']","['hair', 'ply', 'vaccination']",subjective,2,"[ki, raja, se, spray, logon, ne, ape, pardon, ..."
2,deaths due covid what is vaccination rate what...,"[death, due, couid, vaccination, rate, free, s...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2,"[death, due, rate, free, safe, sir, gallian, n..."
3,vaccination against covid- 19 is necessity tod...,"[vaccination, couid, nineteen, necessity, toda...","['necessity', 'today', 'scenario', 'hope', 'pr...","['necessity', 'today', 'scenario', 'problem']",subjective,2,"[nineteen, necessity, today, scenario, hoping,..."
5,dr. deserved #1 minister's award his exception...,"[deserved, one, minister, award, exceptional, ...","['deserved', 'way', 'manage', 'thing', 'millen...","['way', 'thing', 'millennium', 'peril', 'couid...",subjective,2,"[deserved, one, minister, award, exceptional, ..."
6,"yeah i still do not get why, symptoms are much...","[yeah, still, not, get, symptom, much, le, sev...","['still', 'symptom', 'much', 'less', 'severe',...","['symptom', 'vaccination', 'example', 'country...",subjective,2,"[yeah, still, not, get, symptom, much, le, sev..."
...,...,...,...,...,...,...,...
37618,nature seems have already started working savi...,"[nature, seem, already, start, work, save, nex...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'generation', 'vaccinesaveslive', '...",subjective,2,"[nature, seem, already, start, work, save, nex..."
37619,"dear , we #india Indiahas helped this #world i...","[dear, india, indians, help, world, include, u...","['dear', 'help', 'world', 'include', 'kindness...","['world', 'kindnessmatter', 'vaccinesaveslive']",subjective,2,"[dear, india, indians, help, world, include, u..."
37620,our sincere request state central govt folded_...,"[sincere, request, state, central, govt, pleas...","['sincere', 'request', 'state', 'central', 'go...","['request', 'state', 'govt', 'share', 'vaccine...",subjective,2,"[sincere, request, state, central, govt, pleas..."
37621,thanks your appreciation valuable feedback abo...,"[thank, appreciation, valuable, feedback]","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'feedback', 'vaccine...",subjective,2,"[thank, appreciation, valuable, feedback]"


In [28]:
texts = positive_df['words_processed_all']

In [54]:
new_text_all = []

for row in texts:
    # print(type(row))
    new_row = " ".join(row)
    new_text_all.append(new_row)


In [55]:
new_text_all

['couid ki raja se spray logon ne ape pardon ko khoja hai sare log ani jan se hath bethe agr ap susan se bacha chaste hair ply vaccination larva covidvaccine',
 'death due couid vaccination rate free safe sir gallian nikola ata tuck congress mubarak kashmir nha se kashmiri pandit ko braga mar gay harm parivartan rne bola gay wo mubarak dunny ke mubarak nai koi mubarak kashmir nha se kashmiri pandit ko braga mar gay harm',
 'vaccination couid nineteen necessity today scenario hoping problem end soon everyone remains safe secure',
 'deserved one minister award exceptional performance health ministry way managed thing millennium biggest peril couid outstanding amazing job vaccination process retweet agree',
 'yeah still not get symptom much le severe vaccination example country number death much lower seventy population vaccinated couid life back normal',
 'kindly make mutual agreement couid vaccination recognition international travel bow india amp italy many student like need travel res

In [56]:
positive_df['new_text_all'] = new_text_all

/var/folders/lg/w8d05wzd48q3m5_s46qt2fxc0000gn/T/ipykernel_83314/4226431711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_df['new_text_all'] = new_text_all


In [57]:
texts = positive_df['new_text_all']

In [58]:
texts

1        couid ki raja se spray logon ne ape pardon ko ...
2        death due couid vaccination rate free safe sir...
3        vaccination couid nineteen necessity today sce...
5        deserved one minister award exceptional perfor...
6        yeah still not get symptom much le severe vacc...
                               ...                        
37618    nature seem already start work save next gener...
37619    dear india indians help world include usa need...
37620    sincere request state central govt please retw...
37621                 thank appreciation valuable feedback
37622    wait get dose covidvaccine amp do still contin...
Name: new_text_all, Length: 17594, dtype: object

In [59]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

high_freq_known_words = ['covid','couid', 'pfizer', 'amp', 'people', 'covidvaccine', 'covishield', 'moderna', 'vaccine', 'astrazeneca', 'vaccinate', 'vaccinated', 'vaccination', 'vaccines']

stop_words.extend(high_freq_known_words)

In [61]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=4, M=20, alpha=50/4, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [01:13<00:00,  8.19it/s]


In [62]:
stable_topics = [0, 1, 2, 3]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [63]:
top_words

,topic0,topic1,topic2,topic3
0,get,india,hai,get
1,thank,get,ki,one
2,dose,nineteen,se,dose
3,take,country,ke,fully
4,safe,one,nazi,please
5,today,make,ka,two
6,nineteen,world,ko,day
7,first,indian,hi,take
8,please,good,ho,go
9,everyone,free,aur,time


## 5 topics

In [64]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=5, M=20, alpha=50/5, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [01:20<00:00,  7.44it/s]


In [65]:
stable_topics = [0, 1, 2, 3, 4]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [66]:
top_words

,topic0,topic1,topic2,topic3,topic4
0,india,get,get,hai,get
1,get,thank,one,ki,safe
2,country,dose,fully,se,nineteen
3,one,please,two,ke,take
4,nineteen,today,day,nazi,stay
5,make,sir,go,ka,virus
6,indian,take,dose,ko,mask
7,world,help,take,hi,keep
8,free,first,good,ho,corona
9,good,center,time,aur,let


## 6 topics

In [67]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=6, M=20, alpha=50/6, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [01:35<00:00,  6.30it/s]


In [68]:
stable_topics = [0, 1, 2, 3, 4, 5]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [69]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5
0,india,get,get,hai,get,get
1,get,thank,safe,ki,two,one
2,country,dose,nineteen,se,one,fully
3,nineteen,today,take,ke,go,please
4,one,take,stay,nazi,want,day
5,world,please,mask,ka,fully,dose
6,indian,sir,virus,ko,dose,two
7,make,first,keep,hi,good,go
8,free,help,let,ho,take,take
9,good,center,protect,aur,day,even


## 7 topics

In [70]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=7, M=20, alpha=50/7, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [01:38<00:00,  6.12it/s]


In [71]:
stable_topics = [0, 1, 2, 3, 4, 5, 6]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [72]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6
0,one,get,get,india,hai,get,get
1,get,dose,thank,get,ki,fully,safe
2,nineteen,two,dose,country,se,please,nineteen
3,take,one,today,one,ke,one,take
4,virus,go,take,world,nazi,day,stay
5,good,first,center,make,ka,go,please
6,also,take,sir,indian,ko,sir,keep
7,two,want,first,nineteen,hi,help,everyone
8,time,fully,help,free,ho,dose,let
9,still,day,please,good,aur,take,mask


## 8 topics

In [73]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=8, M=20, alpha=50/8, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [01:56<00:00,  5.13it/s]


In [74]:
stable_topics = [0, 1, 2, 3, 4, 5, 6, 7]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [75]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7
0,india,get,get,get,hai,get,get,get
1,country,la,thank,safe,ki,dose,free,please
2,one,tak,dose,nineteen,se,one,india,sir
3,world,fully,today,take,nazi,two,take,fully
4,get,ni,first,stay,ke,day,state,help
5,nineteen,tu,take,mask,ko,take,make,one
6,indian,want,nineteen,virus,ka,go,like,dose
7,make,alhamdulillah,center,keep,hi,good,give,go
8,good,mom,sir,wear_mask,ho,first,govt,need
9,great,je,please,protect,aur,fully,one,travel
